---
title: "Mapping Transit"
author: "Varun Bhakhri, Riya Saini"
format:
  html:
    code-fold: true
jupyter: python3
---




## **Mapping Transit Network**


In [ ]:
#Load Packages:

import os
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import osmnx as ox
import shapely
from shapely.geometry import Point
import folium
from folium import Choropleth

### Load and Inspect GTFS data for Philadelphia

We utilized the General Transit Feed Specification (GTFS), a standardized data format that provides public transportation schedules and associated network information, to extract SEPTA bus stop locations and stop times at each stop. We simultaneously, also import the bus network for the region and clip it to Philadelphia Metropolitan Area city limits for our analysis.


In [ ]:
#Load GTFS Data:

gtfs_folder = "Data/google_bus_Fall 2023"

# Dictionary to store dataframes for each GTFS file
gtfs_dataframes = {}

# Loop through all files in the folder
for file_name in os.listdir(gtfs_folder):
    # Check if the file is a .txt file
    if file_name.endswith(".txt"):
        file_path = os.path.join(gtfs_folder, file_name)
        try:
            # Read the .txt file into a Pandas DataFrame
            dataframe_name = file_name.replace(".txt", "")
            gtfs_dataframes[dataframe_name] = pd.read_csv(file_path)
            print(f"Imported {file_name} as '{dataframe_name}' with {len(gtfs_dataframes[dataframe_name])} rows.")
        except Exception as e:
            print(f"Failed to load {file_name}: {e}")

print(f"Loaded DataFrames: {list(gtfs_dataframes.keys())}")

In [ ]:
# Extract relevant files as dataframes

stops_df = gtfs_dataframes.get("stops", None)
stop_times_df = gtfs_dataframes.get("stop_times", None)
routes_df = gtfs_dataframes.get("routes", None)
shapes_df = gtfs_dataframes.get("shapes", None)

# view files
stops_df.head()
print(shapes_df)

In [ ]:
from shapely.geometry import LineString

# Ensure the DataFrame is sorted by shape_id and shape_pt_sequence
shapes_df = shapes_df.sort_values(by=['shape_id', 'shape_pt_sequence'])

# Group by shape_id and create LineString for each group
lines = (
    shapes_df.groupby('shape_id')
    .apply(lambda group: LineString(zip(group['shape_pt_lon'], group['shape_pt_lat'])))
    .reset_index(name='geometry'))

# Create a GeoDataFrame from the lines
bus_network = gpd.GeoDataFrame(lines, geometry='geometry', crs='EPSG:4326')

bus_network.plot()

# Add city boundary:
city_boundary = ox.geocode_to_gdf("Philadelphia, Pennsylvania, USA")

# Trim Bus network
bus_network_philadelphia = gpd.clip(bus_network, city_boundary)

bus_network_philadelphia.plot()

At this step, we ensure that our segment data is working as desired so that it can combine with our points data.


In [ ]:
from shapely.geometry import Point, LineString, MultiLineString

# Function to convert MultiLineString to LineString
def convert_to_linestring(geometry):
    if isinstance(geometry, MultiLineString):
        # Combine all components into a single LineString
        combined_coords = []
        for line in geometry.geoms:  # Use .geoms to iterate over the components
            combined_coords.extend(line.coords)
        return LineString(combined_coords)
    return geometry  # If already a LineString, return as is

# Apply the conversion function to the GeoDataFrame
bus_network_philadelphia['geometry'] = bus_network_philadelphia['geometry'].apply(convert_to_linestring)

import networkx as nx
from shapely.ops import split, unary_union

# Create a graph from the GeoDataFrame
def geodataframe_to_graph(gdf):
    G = nx.Graph()
    for line in gdf.geometry:
        if isinstance(line, LineString):
            coords = list(line.coords)
            for i in range(len(coords) - 1):
                G.add_edge(Point(coords[i]), Point(coords[i + 1]), geometry=LineString(coords[i:i + 2]))
    return G

# Convert GeoDataFrame to a graph
graph = geodataframe_to_graph(bus_network_philadelphia)

# Split the network into edges divided by nodes
edges = []
for edge in graph.edges(data=True):
    line = edge[2]['geometry']
    start_node = edge[0]
    end_node = edge[1]
    edges.append({
        'geometry': line,
        'start': start_node,
        'end': end_node
    })

# Create a GeoDataFrame of edges
edges_gdf = gpd.GeoDataFrame(edges, crs=bus_network_philadelphia.crs)

edges_gdf = gpd.clip(edges_gdf, city_boundary)

edges_gdf.plot()

### Bus Frequency at Bus Stops

By aggregating this data, we estimated the daily arrivals at each bus stop, effectively quantifying the daily traffic across the network. The distribution, as illustrated in the bar plot below, reveals that while a small number of stops experience exceptionally high traffic exceeding 2,000 buses per day (likely transit interchange hubs), the majority of bus stops accommodate approximately 183 buses daily.


In [ ]:
daily_buses = stop_times_df.groupby(["stop_id"]).size().reset_index(name="daily_arrivals")

daily_buses_sorted = daily_buses.sort_values(by="daily_arrivals", ascending=False).reset_index(drop=True)

daily_buses_sorted.head()

plt.figure(figsize=(8, 6))

# Create histogram 
plt.hist(
    daily_buses["daily_arrivals"],
    bins=range(0, daily_buses["daily_arrivals"].max() + 100, 100),  # Adjust bins
    edgecolor="black",
    align="left"
)

plt.title("Distribution of Bus Stops by Total Daily Bus Arrivals", fontsize=16)
plt.xlabel("Number of Daily Bus Arrivals", fontsize=12)
plt.ylabel("Number of Bus Stops", fontsize=12)

plt.xticks(range(0, daily_buses["daily_arrivals"].max() + 400, 400))
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()

### Total Arrivals and Peak Period

The data was subsequently grouped by each hour of the day to analyze the temporal distribution of bus service. The bar chart below illustrates the hourly distribution, revealing two distinct periods of heightened bus frequency: between 7 AM and 8 AM, and from 3 PM to 6 PM, with consistently elevated service levels during the midday hours. Bus frequency gradually declines after 8 PM until 5 AM. For the purpose of this analysis, the 3 PM to 6 PM window is designated as the peak period due to its sustained high frequency and significance in transit operations.


In [ ]:
stop_times_df["hour"] = stop_times_df["arrival_time"].str.slice(0, 2).astype(int)

# Group by trip_id and hour, and count rows
hourly_bus_arrivals = stop_times_df.groupby(["stop_id", "hour"]).size().reset_index(name="hourly_arrivals")

hourly_bus_arrivals_sorted = hourly_bus_arrivals.sort_values(by="hourly_arrivals", ascending=False).reset_index(drop=True)

hourly_bus_arrivals_sorted.head()

# Find number of buses per hour on the entire system to determine peak period:

total_hourly_buses = stop_times_df.groupby(["hour"]).size().reset_index(name="total_bus_arrivals")

total_hourly_buses = total_hourly_buses[total_hourly_buses["hour"] <= 24]

total_hourly_buses = total_hourly_buses.sort_values(by="total_bus_arrivals", ascending=False).reset_index(drop=True)

total_hourly_buses.head()

In [ ]:
# plot histogram of peak period
plt.figure(figsize=(8, 6))
plt.bar(total_hourly_buses["hour"], total_hourly_buses["total_bus_arrivals"], edgecolor="black")

plt.title("Total Bus Arrivals by Hour", fontsize=16)
plt.xlabel("Hour", fontsize=12)
plt.ylabel("Total Daily Bus Arrivals", fontsize=12)
plt.xticks(total_hourly_buses["hour"])  
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()

### Frequency and Headway

In transit planning, frequency refers to the number of buses servicing a particular stop within a given time frame, typically measured as buses per hour. It is a critical metric for understanding the level of service provided by a transit network, as higher frequency generally translates to shorter wait times for passengers and improved overall service reliability. In this analysis, frequency is calculated by determining the number of bus arrivals at each stop during specified time periods (3 PM to 6 PM).


In [ ]:
peak_bus_arrivals = hourly_bus_arrivals[hourly_bus_arrivals["hour"].isin([3, 4, 5])]

peak_frequency = peak_bus_arrivals.groupby("stop_id", as_index=False)["hourly_arrivals"].sum()

peak_frequency = peak_frequency.rename(columns={"hourly_arrivals": "bus_arrivals"})

# Create a new column 'frequency'
peak_frequency["frequency"] = peak_frequency["bus_arrivals"] / 3

peak_frequency.head()

Additionally, headways—the time interval between consecutive buses at a stop—are directly derived from frequency. It directly impacts passenger experience, as shorter headways result in reduced waiting times and greater convenience, particularly during peak periods when demand is highest. Here, we first calculate the number of bus arrivals at a stop during peak period. After getting the per hour frequency, we calculate the headway at a bus stop by dividing 60 minutes. On an average the headway is 30 minutes.


In [ ]:
# Calculate the headway and assign it to a new column
peak_frequency["headway"] = 60 / peak_frequency["frequency"]

peak_frequency = peak_frequency.sort_values(by="headway", ascending=True).reset_index(drop=True)
print(peak_frequency)

### Ridership

We then merge the ridership at each stop to our existing dataset containing frequency and headway.


In [ ]:
peak_metrics = pd.merge(peak_frequency, stops_df, on="stop_id", how="inner")
ridership = pd.read_csv("Data/Fall_2023_Stop_Summary_(Bus).csv")

ridership["Ridership"] = ridership["Weekday_On"] + ridership["Weekday_Of"]

ridership.rename(columns={"Stop_Code": "stop_id"}, inplace=True)

ridership_by_stop = ridership.groupby("stop_id")["Ridership"].sum().reset_index()

ridership_by_stop.head()

#Merge Ridership and Schedules

peak_metrics = pd.merge(peak_metrics, ridership_by_stop, on="stop_id", how="inner")
print(peak_metrics)

Our analysis aimed to determine whether higher frequency (or shorter headways) correlates with greater ridership. While ridership and frequency exhibit a positive correlation, the presence of many outliers highlights the need for a combined analysis of both variables. This is crucial for identifying streets that would benefit most from improvements such as bus lanes, and it suggests that increasing frequency may be necessary to better accommodate the current ridership levels.


In [ ]:
import numpy as np

# keep numeric
peak_metrics_cleaned = peak_metrics.dropna(subset=['frequency', 'Ridership'])
x = peak_metrics_cleaned['frequency']
y = peak_metrics_cleaned['Ridership']

# Fit a linear regression model
coefficients = np.polyfit(x, y, 1)  
trendline = np.poly1d(coefficients)

# Create scatterplot
plt.figure(figsize=(8, 6))
plt.scatter(x, y, alpha=0.7, label='Data')
plt.plot(x, trendline(x), color='red', linestyle='--', label='Trend Line')

plt.title('Scatterplot of Frequency vs Ridership with Trend Line', fontsize=16)
plt.xlabel('Frequency', fontsize=12)
plt.ylabel('Ridership', fontsize=12)
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

This presumption is supported by the scatter-plot below, which reveals that stops with extremely high ridership are concentrated near the origin, corresponding to headways of under 25 minutes. We can also see that areas with greater headways have their ridership closer to origin well- suggesting lower ridership demand.


In [ ]:
# Scatterplot: Ridership vs. Headway
plt.figure(figsize=(8, 6))

plt.scatter(
    peak_metrics["headway"],
    peak_metrics["Ridership"],
    c="blue",
    alpha=0.4,)

plt.title("Scatterplot: Ridership vs. Headway", fontsize=16)
plt.xlabel("Headway (minutes)", fontsize=12)
plt.ylabel("Ridership", fontsize=12)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

### Level of Service Analysis

To better understand the spatial distribution of bus network, we first clipped the network to city limits.


In [ ]:
geometry = [Point(xy) for xy in zip(peak_metrics["stop_lon"], peak_metrics["stop_lat"])]
peak_metrics_geo = gpd.GeoDataFrame(peak_metrics, geometry=geometry)

# Set a Coordinate Reference System (CRS)
peak_metrics_geo.set_crs(epsg=4326, inplace=True)  # WGS84 CRS

peak_metrics_geo.head()

peak_metrics_geo = gpd.clip(peak_metrics_geo, city_boundary)

To evaluate the level of service across Philadelphia and understand user patterns, we visualized three key components—frequency, headway, and ridership—on a map for peak periods. The map reveals that most areas in the city benefit from frequent bus services, with frequency under 10 during peak times and averaging below 20 minutes headway across the day. However, there are extreme outliers in some regions, where headway exceed 180 minutes, highlighting disparities in service provision. By adding a geographic component, we observe a more nuanced distribution of ridership across Philadelphia. As expected, areas in Center City exhibit high ridership and frequent bus service. Conversely, far-flung areas such as Upper Northwest and Lower Northwest show lower ridership levels and greater headway. Interestingly, Lower Northeast stands out with the highest ridership of all districts.

Interestingly, while frequency and headway vary significantly across the city, ridership appears more evenly distributed, with notable exceptions in Center City, which experiences concentrated high ridership, and the Lower Northwest, near Mt. Airy, which shows higher ridership despite longer headway.


In [ ]:
import ipywidgets as widgets

# create function
def create_map(selected_metric):
    fig, ax = plt.subplots(figsize=(8, 8))
    peak_metrics_geo.plot(
        ax=ax,
        column=selected_metric, 
        cmap="plasma",            
        legend=True,              
        markersize=2)
    
    
    plt.title(f"{selected_metric.capitalize()} by Bus Stop", fontsize=16)
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.show()

dropdown = widgets.Dropdown(
    options=['frequency', 'headway', 'Ridership'],  
    value='frequency',
    description='Metric:',)

widgets.interactive(create_map, selected_metric=dropdown)

### Bus Routes Analysis

#### Spatial Join: Bus Stops to Bus Network

Till now we were looking at transit data at a point, however to understand what routes generate the most traffic and what routes to prioritze, we need to look at the data at street level. To achieve this, we joined the bus stops to the bus network, ensuring that each point is associated with the nearest street segment.


In [ ]:
# Check CRS
peak_metrics_geo = peak_metrics_geo.to_crs("EPSG: 2272")
bus_network_philadelphia = bus_network_philadelphia.to_crs("EPSG: 2272")

peak_metrics_geo = peak_metrics_geo.reset_index(drop=True)

# Perform a spatial join to find the nearest road segment
peak_metrics_network = gpd.sjoin_nearest(
    bus_network_philadelphia,                     
    peak_metrics_geo,  
    how="left",                
    distance_col="distance")

#Re-project to a geographic CRS
peak_metrics_network = peak_metrics_network.to_crs("EPSG: 4326")

min_distance_indices = peak_metrics_network.groupby("stop_id")["distance"].idxmin()

# Filter to keep only the rows with the smallest distance
peak_metrics_network = peak_metrics_network.loc[min_distance_indices]
peak_metrics_network.head()

### Buses at Peak Hour

Following the spatial join, we can now analyze bus headways during peak hours by street segments, providing a more refined view of service levels across different parts of the city. Major corridors in Philadelphia, such as Broad Street, Walnut Street, Chestnut Street, Market Street, and Roosevelt Boulevard, exhibit headways of under 20 minutes, reflecting frequent service. In contrast, streets on the city's periphery have significantly longer headways, exceeding 120 minutes, highlighting areas with less frequent service.


In [ ]:
# headway by street segment
fig, ax = plt.subplots(figsize=(8, 8))
peak_metrics_network.plot(
    ax=ax,
    column="headway",  # Color by buses_count
    cmap="plasma",        # Use a colormap (e.g., viridis, plasma, etc.)
    legend=True,           # Add a legend
    markersize=1         # Adjust marker size
)

# Add titles and labels
plt.title("Headway by Street Segment", fontsize=16)
plt.xlabel("Longitude")
plt.ylabel("Latitude")

When examining ridership by street segments, the distribution tells a different story. Ridership remains relatively consistent across most of the network, regardless of the headway. However, ridership spikes above 2,000 passengers for specific routes around Mt. Airy, Frankford, N Broad St., as well as Center City. These areas, particularly in the North, host major transportation hubs that attract a high volume of trips.


In [ ]:
# Ridership by street segment
fig, ax = plt.subplots(figsize=(8, 8))
peak_metrics_network.plot(
    ax=ax,
    column="Ridership",  # Color by buses_count
    cmap="plasma",        # Use a colormap (e.g., viridis, plasma, etc.)
    legend=True,           # Add a legend
    markersize=1         # Adjust marker size
)

# Add titles and labels
plt.title("Ridership by Street Segment", fontsize=16)
plt.xlabel("Longitude")
plt.ylabel("Latitude")